<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Разлив нефти

### Examples - Fluid Mechanics
<section class="post-meta">
By Tor Nordam, Jonas Blomberg Ghini, Jon Andreas Støvneng, Eilif Sommer Øyre and Jonas Tjemsland
</section>
Last edited: March 22nd 2018

This notebook is based on a project assignment [[1]](#rsc) given in the course TMA4320 Introduction to Scientific Computation at NTNU, spring 2017.
___

## Вступление

Вероятно вам известно, что Норвежский метеорологический институт составляет прогноз погоды, предсказывая погоду на несколько дней вперед. Этот прогноз включает векторное поле скоростей воздуха на высоте 10 м над землей, обычно известное как “ветер”. Менее известно, что сотрудники института также дают прогноз для океана, который включает поле скоростей, описывающее океанские течения примерно на два дня вперед. В случае разлива нефти в море эта информация может быть использована для прогнозирования того, где в конечном итоге окажется нефть, что, в свою очередь, может быть использовано для непосредственного реагирования, чтобы попытаться минимизировать ущерб. Например, во время разлива нефти Deepwater Horizon в Мексиканском заливе в 2010 году ежедневно использовалось численное моделирование, чтобы предсказать, что произойдет в ближайшие несколько дней. 

Цель этого урока - смоделировать перенос вещества океанскими течениями. Хотя разливы нефти в море относительно хорошо известны по освещению в средствах массовой информации, смоделировать то, что происходит, отнюдь не простая задача. Нефть, разлитая в море, проявляет довольно сложное поведение: она может образовывать капли, погруженные в воду, сплошные пятна на поверхности, она частично растворяется, частично испаряется и частично разлагается биологически, и она может образовывать стабильные эмульсии "масло в воде", которые значительно изменят ее свойства. По этим причинам мы рассмотрим более простой случай переноса растворенного химического вещества. Растворенное химическое вещество будет двигаться так же, как и окружающая вода, не опускаясь и не поднимаясь из-за различий в плотности.

Мы будем изучать перенос растворенных химических веществ в океане с помощью метода частиц. Мы будем считывать данные, предоставленные Норвежским метеорологическим институтом (MET), интерполировать эти данные и использовать их для расчета траекторий. Мы также нанесем позиции и поля концентрации на карту, используя различные пакеты библиотек python.

## Предпосылки и теория

В наших симуляциях мы будем представлять растворенную материю в виде частиц, также называемых “элементами Лагранжа”. Идея состоит в том, что частица будет представлять заданное количество растворенного вещества. Это следует интерпретировать не как реальную физическую частицу, а просто как численное приближение. Если у нас есть большое количество частиц, их можно использовать для расчета концентрации, которая пропорциональна плотности частиц (то есть количеству частиц в объеме).

Мы предполагаем, что присутствие частиц не влияет на движение воды, поэтому мы можем принять скорость воды как данность.

Мы также предполагаем, что частица всегда движется с той же скоростью, что и вода. Это, по сути, то же самое, что сказать, что частица не обладает инерцией. В этой модели, если вы придадите частице некоторую скорость и отпустите ее, движение немедленно затухнет из-за трения о воду. Поэтому это движение называется передемпфированным. Эта модель описывается ОДУ

\begin{equation}
\dot{\textbf{x}} = \textbf{v}_w(\textbf{x}, t),
\label{eq:1} \quad (1)
\end{equation}

где $\dot{\textbf{x}}$ - вектор скорости частицы, а $\textbf{v}_w(\textbf{x}, t)$ - скорость (течение) воды в положении $\textbf{x}$ и время $t$ .

Когда коэффициент сопротивления между водой и частицей велик, передемпфированный предел является хорошим приближением.

## Океанографические данные

В этом примере мы будем использовать предварительно рассчитанные данные о течении океана, чтобы определить скорость воды, $\textbf{v}_w$, в зависимости от $\textbf{x}$ и $t$. Океанографические данные получены компанией MET и являются результатами работы механизма численного моделирования, известного как ROM, в области моделей, известной как NorKyst800m. Он предоставляет информацию о текущих скоростях, температуре воды и солености для области, охватывающей все побережье Норвегии, с горизонтальным разрешением 800 × 800 м, с 12 вертикальными слоями и временным разрешением 1 час. 

Данные доступны для загрузки в виде файлов NetCDF [[2]](#rsc), или к данным можно получить доступ через OPeNDAP [[3]](#rsc). NetCDF - это формат файла для хранения данных в массивах. Он довольно хорошо подходит для средних и больших объемов данных (до 100 гигабайт в одном файле отлично работает), и он очень часто используется для географических данных, таких как данные об океане или атмосфере. Для доступа к данным мы будем использовать библиотеку python `xarray` [[4]](#rsc). 

Файлы данных содержат компоненты $x$ и $y$ поля скорости, хранящиеся в двух переменных с именами $u$ и $v$ (довольно часто хранятся только горизонтальные компоненты текущей скорости). Эти переменные хранятся в виде массивов ранга 4, которые задают их значения в зависимости от времени, глубины, положения y и положения x (обратите внимание, что порядок измерений в файлах равен $(t, z, y, x)$). Координаты точек сетки вдоль этих измерений также хранятся в файлах данных в переменных $time$, $depth$, $Y$ и $X$. Для простоты мы проигнорируем измерение глубины, имея дело только с движением в горизонтальной плоскости и временем.

На рис. [1](#im1) приведены данные о температуре поверхностных вод в качестве примера, иллюстрирующего объем доступных данных. На рис. [2](#im2), те же данные отображаются в системе координат, используемой в файле, с координатами $x$ по горизонтальной оси и координатами $y$ по вертикали. Начало системы координат находится на Северном полюсе.

Размеры $x$ и $y$, как показано на рис. [2](#im2), являются координатными осями в так называемой полярной стереографической проекции поверхности Земли на плоскость. Для целей этого проекта мы будем иметь дело с движением в плоскости $xy$с координатами, как показано на рисунке. Поскольку векторные компоненты поля скоростей выровнены по этим координатным осям, мы можем использовать компоненты непосредственно для вычисления движения в $xy$-плоскости. Это означает, что для моделирования транспорта мы будем игнорировать кривизну Земли. В конце концов, мы увидим, как преобразовать координаты $xy$ в долготу и широту и построить положение частиц на карте.

<a name="im1"></a>
<img src="images/domain.png" width="600">  
**Рисунок 1:** *Область модели NorKyst800m, показывающая температуру поверхностных вод 4 февраля 2017 года.*

<a name="im2"></a>
<img src="images/stereographic_projection.png" width="650">  
**Рисунок 2:** *Область модели NorKyst800m, показанная в системе координат, используемой для хранения данных. Начало координат системы координат находится на Северном полюсе (обозначено ×), а расстояния указаны в метрах.*

<a name="im3"></a>
<img src="images/grid.png" width="350">
**Рисунок 3:** *Компоненты вектора скорости задаются только в дискретных точках, $(x_i, y_j) = (x_0 + i\Delta x, y_0+j\Delta y)$. Чтобы рассчитать траектории, нам нужно оценить скорость в произвольных положениях $(x, y)$* 

## Построение графиков на картах

После расчета того, как частицы переносятся океанскими течениями, мы построим их траектории и положение на карте. Для отображения данных на картах с помощью python используются две основные библиотеки, `basemap` и `cartopy`. Iв этом примере мы будем использовать `cartopy` [[5]](#rsc), потому как она немного проще в использовании. `cartopy` это пакет Python, предназначенный для того, чтобы максимально упростить рисование карт для анализа и визуализации данных. 

При перемещении частиц со скоростью воды мы будем использовать систему координат, показанную на рис. [2](#im2). Для построения траекторий удобно использовать эти координаты напрямую и в метрах. Однако, если мы хотим показать траектории на карте, нам нужно преобразовать систему координат $xy$ полярной стереографической проекции в долготу и широту. Мы будем использовать библиотеку под названием `pyproj` [[6]](#rsc). Этот пакет Python выполняет картографические преобразования и геодезические вычисления. Класс `pyproj.Proj` может преобразовываться из географических (долгота, широта) в собственные координаты проекции карты $(x,y)$ и наоборот, или из одной системы координат проекции карты непосредственно в другую. 

## The Problem

Мы хотим представить перенос растворенного химического вещества в океане, имитируя движение большого количества частиц. Траектория частицы управляется уравнением (1), где $\textbf{v}_w$ берется из данных об океане. Файл данных `Norkyst-800m.nc` содержит данные за 20 дней, охватывающие период с 1 по 20 февраля 2017 года. Файл доступен для скачивания здесь:
* http://folk.ntnu.no/nordam/data/Norkyst-800m.nc

На движение любой одной частицы не влияет присутствие других частиц. Мы начинаем с набора частиц $N_p$, каждая частица в позиции $\textbf{x}_0$ при $t = 0$, случайным образом помещенных в квадрат, определяемый $-3010000 < x < -2990000$, $-1210000 < y < -1190000$. Все частицы переносятся океанским течением в течение 10 дней.

Для распространения частиц мы используем явный метод трапеции в качестве интегратора. Шаг по времени установлен равным $h = 3600$ с, и, поскольку данные предоставляются с интервалом в один час, интерполяция по времени не требуется.

### Задачи

**a.** Нанесите положение частиц на карту при $t = 0$ и после $2$, $4$, $6$, $8$, и $10$ дней.

**b.** Рассчитайте и нанесите концентрации химических веществ на карту после $2$, $4$, $6$, $8$, и $10$ дней.

## Решение

Во-первых, мы импортируем необходимые пакеты python.

In [ ]:
conda install -c conda-forge xarray dask netCDF4 bottleneck

In [ ]:
conda install -c conda-forge cartopy

In [ ]:
conda install -c conda-forge pyproj

In [ ]:
%matplotlib inline
from time import time
from datetime import datetime, timedelta
import numpy as np
from matplotlib import pyplot as plt
# nicer looking default plots
plt.style.use('bmh')

# Library to read data from NetCDF files
import xarray as xr

# 2D spline interpolation routine
from scipy.interpolate import RectBivariateSpline

# Map plotting library
import cartopy
# The subclass CRS contains all types of coordinate
# reference systems. We will mainly be using the 
# projection NorthPolarStereo
import cartopy.crs as ccrs
# The subclass feature represents a collection of points, 
# lines, and polygons with convenience methods for common 
# drawing and filtering operations.
import cartopy.feature as cfeature

# library for coordinate transformations
import pyproj

### Служебные функции

Как представление файла NetCDF в памяти, `Norkyst-800m.nc`, мы используем структуру `xarray.dataset`. Он похож на словарь контейнер помеченных массивов с выровненными размерами. Его интерфейс, похожий на словарь, можно использовать для доступа к любой переменной в наборе данных. Подробнее о `xarray.dataset` [здесь](http://xarray.pydata.org/en/stable/data-structures.html#dataset).

В этом примере компоненты $x$ и $y$ поля скорости хранятся как `u` и `v`, а координаты сетки-как массивы `X` и `Y`. Переменная времени `time` хранится как тип `datatime64`. Массив значений конкретных компонент скорости в координатах сетки $(x_0, y_0), ..., (x_i, y_i)$ возвращается при выполнении ```dataset.u[time, depth, y0:yi, x0:xi]```.


In [ ]:
datapath = "~/downloads/NorKyst-800m.nc"
d  = xr.open_dataset(datapath)

Теперь мы определим несколько полезных функций. 

Чтобы интерполировать скорость $\textbf{v}_w$ в произвольную точку $(x, y)$ в системе координат $xy$, мы используем класс `RectBivariateSpline` из `scipy.interpolate`, как обсуждалось ранее.

In [ ]:
class Interpolator():
    """ Interpolating the datasets velocity components using bivariate spline
    interpolation over a rectangular mesh.
    The memberfunction get_interpolators returns functions for the 
    velocity components' interpolated value at arbitrary positions.
    
    Parameters
    ----------
    dataset : xarray_type
            Data structure containing the oceanographic data.
    X       : array_type
            Particle coordinates.
    t       : datetime64_type
            Time.
    ----------
    """
    def __init__(self, dataset):
        self.dataset = dataset
        
    def get_interpolators(self, X, it):
        # Add a buffer of cells around the extent of the particle cloud
        buf  = 3
        # Find extent of particle cloud in terms of indices
        imax = np.searchsorted(self.dataset.X, np.amax(X[0,:])) + buf
        imin = np.searchsorted(self.dataset.X, np.amin(X[0,:])) - buf
        jmax = np.searchsorted(self.dataset.Y, np.amax(X[1,:])) + buf
        jmin = np.searchsorted(self.dataset.Y, np.amin(X[1,:])) - buf
        # Take out subset of array, to pass to RectBivariateSpline
        # Transpose to get regular order of coordinates (x,y)
        # Fill NaN values (land cells) with 0, otherwise
        # interpolation won't work
        u    = self.dataset.u[it, 0, jmin:jmax, imin:imax].T.fillna(0.0)
        v    = self.dataset.v[it, 0, jmin:jmax, imin:imax].T.fillna(0.0)
        # RectBivariateSpline returns a function-like object,
        # which can be called to get value at arbitrary position
        fu   = RectBivariateSpline(self.dataset.X[imin:imax], self.dataset.Y[jmin:jmax], u)
        fv   = RectBivariateSpline(self.dataset.X[imin:imax], self.dataset.Y[jmin:jmax], v)
        return fu, fv

    def get_time_index(self, t):
        # Get index of largest timestamp smaller than (or equal to) t
        return np.searchsorted(self.dataset.time, t, side='right') - 1

    def __call__(self, X, t):
        # get index of current time in dataset
        it = self.get_time_index(t)
        # get interpolating functions,
        # covering the extent of the particle
        fu, fv = self.get_interpolators(X, it)
        # Evaluate velocity at position(x[:], y[:])
        vx = fu(X[0,:], X[1,:], grid = False)
        vy = fv(X[0,:], X[1,:], grid = False)
        return np.array([vx, vy])

Чтобы решить ОДУ (1), нам нужен интегратор. В этом примере мы используем явный метод трапеции (метод Ранга-Кутты второго порядка).

In [ ]:
def rk2(x, t, h, f):
    """ A second order Rung-Kutta method.
        The Explicit Trapezoid Method.
        
    Parameters:
    -----------
        x :    coordinates (as an array of vectors)
        h :    timestep
        f :    A function that returns the derivatives
    Returns:
        Next coordinates (as an array of vectors)   
    -----------
    """

    # Note: t and h have actual time units.
    # For multiplying with h, we need to
    # convert to number of seconds:
    dt = h / np.timedelta64(1, 's')
    # "Slopes"
    k1  = f(x,           t)
    k2  = f(x + k1*dt,   t + h)
    # Calculate next position
    x_ = x + dt*(k1 + k2)/2
    return x_

Наконец, мы определяем функцию для вычисления траектории частиц.

In [ ]:
def trajectory(X0, t0, Tmax, h, f, integrator):
    """ Function to calculate trajectory of the particles.
    
    Parameters:
    -----------
        X0 :    A two dimensional array containing start positions 
                (x0, y0) of each particle.
        t0 :    Initial time
        Tmax:   Final time
        h  :    Timestep
        f  :    Interpolator
        integrator:   The chosen integrator function
    
    Returns:
        A three dimensional array containing the positions of
        each particle at every timestep on the interval (t0, Tmax).
    -----------
    """
    
    Nt   = int((Tmax-t0) / h)          # Number of datapoints
    X    = np.zeros((Nt+2, *X0.shape))
    X[0,:] = X0
    t = t0
    for i in range(Nt+1):
        # Adjust last timestep to match Tmax exactly
        h  = min(h, Tmax - t)
        t += h
        X[i+1,:] = integrator(X[i,:], t, h, f)
    return X

### Задача a.

- Рассчитайте траекторию частиц $N_p$ и нанесите позиции на карту через 0, 2, 4, 6, 8 и 10 дней.

In [ ]:
# Initialise interpolator with dataset
datapath = "~/downloads/NorKyst-800m.nc"
d  = xr.open_dataset(datapath)
f  = Interpolator(dataset = d)

# Set initial conditions (t0 and x0) and timestep
# Note that h also has time units, for convenient
# calculation of t + h.
h  = np.timedelta64(3600, 's')
# setting X0 in a slightly roundabout manner for
# compatibility with Np >= 1
Np = 10000
X0 = np.zeros((2, Np))
X0[0,:] = np.random.uniform(-3010000, -2990000, size = Np)
X0[1,:] = np.random.uniform(-1210000, -1190000, size = Np)

# Dataset covers 2017-02-01 00:00 to 2017-02-19 23:00
t0   = np.datetime64('2017-02-01T12:00:00')
# Calculate 10 day trajectory
Tmax = t0 + np.timedelta64(10, 'D')
X1   = trajectory(X0, t0, Tmax, h, f, rk2)

Массив `X1` теперь содержит координату $xy$ для каждой из частиц $N_p$ в каждый момент времени $t$. Он имеет размеры `(Nt, 2, Np)`.
Теперь мы можем отображать частицы на карте в любое удобное для нас время, а также подсчитывать концентрацию частиц в определенной сеточной системе. Для создания фигуры со стереографической проекцией мы используем `ccr`, для добавления береговых линий и других объектов мы используем `cfeature`, а для преобразования декартовых координат в долготу и широту мы используем `pyproj`.

In [ ]:
# PLOTTING PARITCLES ON MAP
# Step 1 
# Create a figure object, and add 6 axes instances, with projection info
fig = plt.figure(figsize=(15,10))
axes = []
axes.append(fig.add_subplot(231, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(232, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(233, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(234, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(235, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(236, projection=ccrs.NorthPolarStereo()))

# Step 2:
# It doesn't look like a map unless we add land and sea
# In order to draw land and coastlines, we use built-in functions
# in cartopy.feature. These will download some data from
# www.naturalearthdata.com/ the first time they are called.
# (resolution 10m means 1 : 10,000,000, not 10 meters)
land_10m  = cfeature.NaturalEarthFeature('physical', 'land', '10m', color = '#dddddd')
for ax in axes:
    # Add land and coastline
    ax.add_feature(land_10m)
    ax.coastlines(resolution='10m')

# Create projection with metadata from dataset
# and latlon projection
p1 = pyproj.Proj(d.projection_stere.proj4)
p2 = pyproj.Proj(proj='latlong')

# Step 3:
# Convert coordinates to longitude and latitude
lons, lats = pyproj.transform(p1, p2, X1[:,0,:], X1[:,1,:])

# Step 4:
# Plot data
for i, it in enumerate(np.arange(0, 10*24 + 1, 2*24)): # (start, stop (after 10 days), step (2 days))
    axes[i].set_title("Day " + str(0 + 2*i))
    axes[i].scatter(lons[it,:], lats[it,:], marker = '.', lw = 0, s = 20, alpha = 0.8, 
                    transform=ccrs.Geodetic(), zorder=2)

# Step 5 (optional):
# Set the extent of the map. If we leave out these, it would
# just cover the plotted points, and nothing more. Specify
# (lon0, lon1, lat0, lat1), and Cartopy will make sure the
# map area is large enough to cover the four points
# (lon0, lat0), (lon0, lat1), (lon1, lat0), (lon1, lat1).
for ax in axes:
    ax.set_extent((0, 9, 57.5, 62))

# try to automatically reduce white space in figure
plt.tight_layout()

Как и ожидалось, результаты показывают, что химические вещества воздействуют довольно коллективно, с небольшой деформацией формы в течение первых четырех дней. Мы видим, что группа движется в основном на север. Небольшое распространение указывает лишь на незначительные локальные различия в океанском течении.

### Задача b.

- Рассчитайте концентрацию по сетке, подсчитав частицы в ячейках. 

Координаты частиц уже сохранены в массиве `X1`. Теперь нам нужно только определить сеточную систему и подсчитать количество частиц в каждой ячейке сетки. Последняя хранится в двумерном массиве`counts`, где каждый элемент представляет концентрацию химического вещества в соответствующей ячейке сетки 800×800 м. Затем концентрация иллюстрируется построением четырехугольной сетки с использованием функции `matplotlib.axes` `pcolormesh`. Ячейки сетки с наименьшей концентрацией окрашены в фиолетовый цвет, а сетки с наибольшей концентрацией окрашены в желтый цвет. Массив с масками используется только для построения ячеек с ненулевыми концентрациями.

Функция `plt.pcolormesh()` в основном отображает каждый элемент 2D-массива в виде крошечного прямоугольника. Для каждого крошечного прямоугольника ему нужно знать три вещи: $x$-координата, $y$-координата и значение. Координаты должны быть преобразованы из 1D массивов в 2D массивы той же формы, что и счетчики. Для этого используется `np.meshgrid`. Для подсчета частиц в каждой сетке мы используем функцию numpy `np.histogram2d`. В качестве параметров этой функции необходимы координаты $x$ и $y$ точек, подлежащих гистограмме, а также спецификация ячейки, которая в данном случае является экстентом системы сетки.

In [ ]:
# CALCULATING AND PLOTTING GRIDDED CONCENTRATION
# Creating figure, axes and plotting features as in task a.
fig = plt.figure(figsize=(15, 10))
land_10m  = cfeature.NaturalEarthFeature('physical', 'land', '10m', color='#dddddd')
axes = []
axes.append(fig.add_subplot(231, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(232, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(233, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(234, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(235, projection=ccrs.NorthPolarStereo()))
axes.append(fig.add_subplot(236, projection=ccrs.NorthPolarStereo()))

for ax in axes:
    # Add land and coastline
    ax.add_feature(land_10m)
    ax.coastlines(resolution='10m')
    # Set the extent of the map. (lon0, lon1, lat0, lat1)
    ax.set_extent((1, 5.1, 59.4, 61))

# Create projection with metadata from dataset
# and latlon projection
p1 = pyproj.Proj(d.projection_stere.proj4)
p2 = pyproj.Proj(proj='latlong')

# Plot concentrations on the map.
for i, it in enumerate(np.arange(0, 241, 48)):
    # Particle positions at current time
    x = X1[it, 0, :]
    y = X1[it, 1, :]
    
    # Grid size (m)
    wid = 800
    # Uses the particles position to define the extent of the grid system, for each time.
    Xcoords = np.arange(np.amin(x) - 2*wid, np.amax(x) + 2*wid, wid)
    Ycoords = np.arange(np.amin(y) - 2*wid, np.amax(y) + 2*wid, wid)
    
    # To count particles in each cell, we use numpy.histogram2d
    counts, edgeX, edgeY = np.histogram2d(x, y, bins = (Xcoords, Ycoords))
    # Use bin edges returned by function as coordinates
    edgeX, edgeY = np.meshgrid(edgeX, edgeY)
    # Convert coordinates
    lons, lats = pyproj.transform(p1, p2, edgeX, edgeY)

    # Finally, we have chosen to use a masked array to plot
    # the grid data. This is mainly for visibility, as it avoids
    # plotting those cells where the value is 0.
    # Note that mask = True means that the element is not plotted
    # (because it is hidden behind the mask)
    # We use counts == 0 as the mask
    counts_masked = np.ma.masked_array(counts, mask = counts == 0)
    axes[i].pcolormesh(lons, lats, counts_masked.T, transform=ccrs.PlateCarree(), zorder=2)
    axes[i].set_title("Day " + str(0 + 2*i))

plt.tight_layout()

Чтобы более внимательно посмотреть на концентрацию через десять дней, мы меняем масштаб.

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.NorthPolarStereo())
ax.coastlines(resolution='10m')
# Set extent to match extent of grid system
ax.set_extent((np.amin(lons), np.amax(lons), np.amin(lats), np.amax(lats)))
cbar = ax.pcolormesh(lons, lats, counts_masked.T, transform=ccrs.PlateCarree(), zorder=2)
fig.colorbar(cbar, orientation="vertical", label="Particles per grid cell")
ax.set_title("Day 10")
plt.show()

## Вывод

Теперь мы успешно завершили упрощенное моделирование движения химического вещества в течение десяти дней после разлива, используя представление числовыми частицами и предполагая, что частицы двигались с той же скоростью, что и вода.

Частицы были перенесены океанскими течениями, извлеченными из файла NetCDF и представленными в виде xarray. Компоненты скорости тока в сетке были интерполированы с использованием функции scipy, а траектории численных частиц были рассчитаны с использованием явного метода трапеции в качестве интегратора с шагом по времени в час.

Наконец, местоположение и концентрация нефти были нанесены на карту с использованием пакетов Python `cartopy` и `pyproj`.

<a name="rsc"></a>

## Resources and Further Readings


<a >[1]</a>:  Tor Nordam, Jonas Blomberg Ghini, Jon Andreas Støvneng. *Project Assignment: Particle-based simulation of transport by ocean currents*, 2017.<br />
<a>[2]</a>:   Network common data form, see [www.unidata.ucar.edu/software/netcdf/](http://www.unidata.ucar.edu/software/netcdf/).<br />
<a >[3]</a>:  Open-source Project for a Network Data Access Protocol, see [www.opendap.org](http://www.opendap.org).<br />
<a >[4]</a>:  XArray.dataset doc @[www.xarray.pydata.org](http://xarray.pydata.org/en/stable/data-structures.html#dataset).<br />
<a >[5]</a>:  For more Cartopy reference systems and map axes features, see [cartopy](http://scitools.org.uk/cartopy/docs/v0.13/index.html).<br />
<a >[6]</a>:  Pyproj docs @[github](http://jswhit.github.io/pyproj/).<br />